## Read Data

In [14]:
import pandas as pd
import numpy as np

movies = pd.read_csv("../ml-latest-small/Movie_Data.csv")
ratings = pd.read_pickle("../ml-latest-small/rating.p")
genres = pd.read_pickle("../ml-latest-small/genres.p")

genres.sample()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
60522,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0


### Preprocessing

In [15]:
ratings = ratings.merge(genres, left_on = 'movieId', right_index = True)
ratings = ratings.replace(0, np.nan)

ratings

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,964982703,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,5,1,4.0,847434962,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
874,7,1,4.5,1106635946,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1434,15,1,2.5,1510577970,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1667,17,1,4.5,1305696483,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100820,610,160341,2.5,1479545749,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
100821,610,160527,4.5,1479544998,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100823,610,160836,3.0,1493844794,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
100827,610,163937,3.5,1493848789,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


### Train Test Split

In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(ratings, random_state=42, test_size=.1)

### Item Profile

In [17]:
genres.loc[4896]

(no genres listed)    0
Action                0
Adventure             1
Animation             0
Children              1
Comedy                0
Crime                 0
Documentary           0
Drama                 0
Fantasy               1
Film-Noir             0
Horror                0
IMAX                  0
Musical               0
Mystery               0
Romance               0
Sci-Fi                0
Thriller              0
War                   0
Western               0
Name: 4896, dtype: int64

### User Profile

In [18]:
genres_cols = genres.columns

In [23]:
for cols in genres_cols:
    train[cols] = train[cols] * train['rating']

train.sample(5)

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
24112,167,1307,3.0,1154723169,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
88878,573,30749,3.5,1186589572,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.25,NaN
66666,429,151,4.0,828124616,NaN,16.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,16.00,NaN
37510,253,608,5.0,1286643377,NaN,NaN,NaN,NaN,NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN
52952,348,2019,5.0,1378850189,NaN,25.0,25.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 각 유저별로 아이템에 준 평균

In [20]:
User_Profile = train.groupby('userId')[genres_cols].mean()
User_Profile.sample(5)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
218,NaN,3.166667,3.250000,NaN,NaN,3.454545,3.000000,NaN,4.000000,3.000000,NaN,3.750000,3.000000,3.25,3.000000,3.666667,2.900000,3.000000,4.000000,NaN
344,NaN,4.225806,4.392857,4.571429,4.416667,4.031250,3.642857,NaN,3.400000,4.083333,1.500000,1.250000,3.916667,5.00,2.888889,3.875000,4.134615,3.447368,3.875000,4.750
362,NaN,4.200000,4.275000,4.125000,4.000000,4.071429,4.348837,NaN,4.340000,4.500000,4.333333,4.269231,3.812500,4.25,4.187500,4.100000,4.235294,4.267442,4.300000,4.875
352,NaN,3.545918,3.785714,3.666667,3.200000,3.482143,3.808511,NaN,4.000000,3.615385,3.000000,3.969697,3.984375,3.75,4.102941,3.706897,3.770833,3.765625,4.727273,3.875
119,NaN,4.111111,4.176923,4.145161,4.120000,4.084615,4.280488,3.875,4.292208,3.934783,4.000000,4.000000,4.250000,5.00,4.333333,4.343750,3.985714,4.137097,4.454545,3.875


### My User Profile

In [21]:
User_Profile.loc[400]

(no genres listed)         NaN
Action                4.527778
Adventure             4.269231
Animation             4.500000
Children              4.500000
Comedy                4.666667
Crime                 4.777778
Documentary                NaN
Drama                 4.590909
Fantasy               4.300000
Film-Noir                  NaN
Horror                4.750000
IMAX                  4.200000
Musical                    NaN
Mystery               4.625000
Romance               4.500000
Sci-Fi                4.423077
Thriller              4.666667
War                   4.500000
Western                    NaN
Name: 400, dtype: float64

### Predict

In [22]:
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
predict = list()

for idx, row in test.iterrows():
    user = row['userId']
    predict.append((User_Profile.loc[user] * row[genres_cols]).mean())

In [ ]:
test['predict'] = predict
test.loc[test['predict'].isnull(), 'predict'] = train['rating'].mean()

test['predict'].isna().sum()

0

### Model Validation

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error()